<a href="https://colab.research.google.com/github/aksharat/MLProblems/blob/main/Intro/Data_cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocess the files and import statements
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html <br>

https://stackoverflow.com/questions/27012151/forward-fill-specific-columns-in-pandas-dataframe<br>

Add in the files and import the required packages

In [222]:
!unzip /content/Clicked_DataSprint.zip

Archive:  /content/Clicked_DataSprint.zip
replace data_dictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [64]:
data_dict = pd.read_excel('data_dictionary.xlsx')
emp_data=pd.read_csv('employee_survey_data.csv')
gen_data=pd.read_csv('general_data.csv')
manager_data=pd.read_csv('manager_survey_data.csv')
in_data=pd.read_csv('in_time.csv')
out_data=pd.read_csv('out_time.csv')


In [223]:
def insert_blank_rows_into_data_dict(data_dict, insert_index, num_blank_rows):
    """
    Inserts blank rows into the 'data_dict' DataFrame at the specified index.

    Parameters:
    data_dict (pd.DataFrame): The DataFrame where blank rows will be inserted.
    insert_index (int): The index after which to insert the blank rows.
    num_blank_rows (int): The number of blank rows to insert.

    Returns:
    pd.DataFrame: A DataFrame with the blank rows inserted.
    """
    # Create a DataFrame with blank rows (same structure as data_dict)
    blank_rows = pd.DataFrame({col: [None] * num_blank_rows for col in data_dict.columns})

    # Split the DataFrame into two parts: before and after the insertion point
    df_top = data_dict.iloc[:insert_index+1]  # Up to the insert_index row (inclusive)
    df_bottom = data_dict.iloc[insert_index+1:]  # From the row after insert_index

    # Concatenate the top, blank rows, and bottom together
    data_dict = pd.concat([df_top, blank_rows, df_bottom], ignore_index=True)

    return data_dict

### Data dict analysis

In [65]:
data_dict

,Variable,Meaning,Levels
0,Age,Age of the employee,NaN
1,Attrition,Whether the employee left in the previous year...,NaN
2,BusinessTravel,How frequently the employees travelled for bus...,NaN
3,Department,Department in company,NaN
4,DistanceFromHome,Distance from home in kms,NaN
5,Education,Education Level,1 'Below College'
6,NaN,NaN,2 'College'
7,NaN,NaN,3 'Bachelor'
8,NaN,NaN,4 'Master'
9,NaN,NaN,5 'Doctor'


In [66]:
cols=['Variable','Meaning']
data_dict.loc[:,cols]=data_dict.loc[:,cols].ffill()

In [68]:
# Step 2: Create new columns for level_grade and level_description
def split_levels(level):
    if pd.isnull(level):
        return None, None
    # Split the level to separate the number and the description
    parts = level.split(" ", 1)
    level_grade = parts[0]  # Get the numeric part (level)
    level_description = parts[1].strip("'") if len(parts) > 1 else None  # Get the description part
    return level_grade, level_description

# Apply the function to split the Levels column
data_dict[['level_grade', 'level_description']] = data_dict['Levels'].apply(lambda x: pd.Series(split_levels(x)))



In [69]:
data_dict.iloc[2]['Meaning']

'How frequently the employees travelled for business purposes in the last year'

### Employee data

In [72]:
emp_data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [73]:
emp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB


In [74]:
# Create the SimpleImputer to fill missing values with the mode (most frequent value)
imputer = SimpleImputer(strategy='most_frequent')
emp_data[:] = imputer.fit_transform(emp_data)
columns_to_impute = ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']
# Step 4: Convert the columns from float to int
emp_data[columns_to_impute] = emp_data[columns_to_impute].astype(int)

In [78]:
emp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   EmployeeID               4410 non-null   int64
 1   EnvironmentSatisfaction  4410 non-null   int64
 2   JobSatisfaction          4410 non-null   int64
 3   WorkLifeBalance          4410 non-null   int64
dtypes: int64(4)
memory usage: 137.9 KB


### General data

In [79]:
gen_data.head()


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [85]:
gen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

#### Data dict updates

##### Age

In [83]:
gen_data['Age'].unique()

array([51, 31, 32, 38, 46, 28, 29, 25, 45, 36, 55, 47, 37, 21, 35, 26, 50,
       53, 42, 44, 49, 18, 41, 39, 58, 33, 43, 52, 27, 30, 54, 40, 23, 48,
       57, 34, 24, 22, 56, 60, 19, 20, 59])

In [84]:
data_dict.loc[0,['Levels', 'level_grade','level_description']] = ["0 'Numeric-Age'", 0,"Numeric-Age"]

##### Attrition

In [81]:
gen_data['Attrition'].unique()

array(['No', 'Yes'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=1, num_blank_rows=1)

In [93]:
data_dict.loc[1,['Levels', 'level_grade','level_description']] = ["1 'No'", 1,"No"]
data_dict.loc[2] = ["Attrition","Whether the employee left in the previous year or not","2 'Yes'", 2,"Yes"]

##### BusinessTravel

In [80]:
gen_data['BusinessTravel'].unique()

array(['Travel_Rarely', 'Travel_Frequently', 'Non-Travel'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=3, num_blank_rows=2)

In [88]:
data_dict.loc[2,['Levels', 'level_grade','level_description']] = ["1 'Non-Travel'", 1,"Non-Travel"]
data_dict.loc[4] = ["BusinessTravel","How frequently the employees travelled for business purposes in the last year","3 'Travel_Frequently'", 3,"Travel_Frequently"]
data_dict.loc[3] = ["BusinessTravel","How frequently the employees travelled for business purposes in the last year","2 'Travel_Rarely'", 2,"Travel_Rarely"]

##### Department

In [82]:
gen_data['Department'].unique()

array(['Sales', 'Research & Development', 'Human Resources'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=6, num_blank_rows=2)

In [99]:
data_dict.loc[6,['Levels', 'level_grade','level_description']] = ["1 'Sales'", 1,"Sales"]
data_dict.loc[7] = ["Department","Department in company","2 'Research & Development'", 2,"Research & Development"]
data_dict.loc[8] = ["Department","Department in company","3 'Human Resources'", 3,"Human Resources"]

##### DistanceFromHome

In [101]:
gen_data['DistanceFromHome'].unique()

array([ 6, 10, 17,  2,  8, 11, 18,  1,  7, 28, 14,  3,  4, 16,  9,  5, 20,
       29, 15, 13, 24, 19, 22, 25, 21, 26, 27, 12, 23])

In [102]:
data_dict.loc[9,['Levels', 'level_grade','level_description']] = ["0 'Distance'", 0,"Distance"]

##### EducationField

In [104]:
gen_data['EducationField'].unique()

array(['Life Sciences', 'Other', 'Medical', 'Marketing',
       'Technical Degree', 'Human Resources'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=15, num_blank_rows=5)

In [110]:
data_dict.loc[15,['Levels', 'level_grade','level_description']] = ["1 'Life Sciences'", 1,"Life Sciences"]
data_dict.loc[16] = ["EducationField","Field of education","2 'Medical'", 2,"Medical"]
data_dict.loc[17] = ["EducationField","Field of education","3 'Human Resources'", 3,"Human Resources"]
data_dict.loc[18] = ["EducationField","Field of education","4 'Marketing'", 4,"Marketing"]
data_dict.loc[19] = ["EducationField","Field of education","5 'Technical Degree'", 5,"Technical Degree"]
data_dict.loc[20] = ["EducationField","Field of education","6 'Other'", 6,"Other"]

##### EmployeeCount

In [112]:
gen_data['EmployeeCount'].unique()

array([1])

In [114]:
data_dict.loc[21,['Levels', 'level_grade','level_description']] = ["1 'UniqueEmployees'", 1,"UniqueEmployees"]

##### EmployeeID

In [113]:
gen_data['EmployeeID'].unique()

array([   1,    2,    3, ..., 4408, 4409, 4410])

In [120]:
data_dict.loc[22,['Levels', 'level_grade','level_description']] = ["0 'Emp-ID'", 0,"Emp-ID"]

##### Gender

In [122]:
gen_data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=27, num_blank_rows=1)

In [125]:
data_dict.loc[27,['Levels', 'level_grade','level_description']] = ["1 'Female'", 1,"Female"]
data_dict.loc[28] = ["Gender","Gender of employee","2 'Male'", 2,"Male"]

##### JobLevel

In [126]:
gen_data['JobLevel'].unique()

array([1, 4, 3, 2, 5])

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=33, num_blank_rows=4)

In [130]:
data_dict.loc[33,['Levels', 'level_grade','level_description']] = ["1 'Level-1'", 1,"Level-1"]
data_dict.loc[34] = ["JobLevel","Job level at company on a scale of 1 to 5","2 'Level-2'", 2,"Level-2"]
data_dict.loc[35] = ["JobLevel","Job level at company on a scale of 1 to 5","3 'Level-3'", 3,"Level-3"]
data_dict.loc[36] = ["JobLevel","Job level at company on a scale of 1 to 5","4 'Level-4'", 4,"Level-4"]
data_dict.loc[37] = ["JobLevel","Job level at company on a scale of 1 to 5","5 'Level-5'", 5,"Level-5"]

##### JobRole

In [132]:
gen_data['JobRole'].unique()

array(['Healthcare Representative', 'Research Scientist',
       'Sales Executive', 'Human Resources', 'Research Director',
       'Laboratory Technician', 'Manufacturing Director',
       'Sales Representative', 'Manager'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=38, num_blank_rows=8)

In [138]:
data_dict.loc[38,['Levels', 'level_grade','level_description']] = ["1 'Healthcare Representative'", 1,"Healthcare Representative"]
data_dict.loc[39] = ["JobRole","Name of job role in company","2 'Research Scientist'", 2,"Research Scientist"]
data_dict.loc[40] = ["JobRole","Name of job role in company","3 'Sales Executive'", 3,"Sales Executive"]
data_dict.loc[41] = ["JobRole","Name of job role in company","4 'Human Resources'", 4,"Human Resources"]
data_dict.loc[42] = ["JobRole","Name of job role in company","5 'Research Director'", 5,"Research Director"]
data_dict.loc[43] = ["JobRole","Name of job role in company","6 'Laboratory Technician'", 6,"Laboratory Technician"]
data_dict.loc[44] = ["JobRole","Name of job role in company","7 'Manufacturing Director'", 7,"Manufacturing Director"]
data_dict.loc[45] = ["JobRole","Name of job role in company","8 'Sales Representative'", 8,"Sales Representative"]
data_dict.loc[46] = ["JobRole","Name of job role in company","9 'Manager'", 9,"Manager"]

##### MaritalStatus

In [140]:
gen_data['MaritalStatus'].unique()

array(['Married', 'Single', 'Divorced'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=51, num_blank_rows=2)

In [143]:
data_dict.loc[51,['Levels', 'level_grade','level_description']] = ["1 'Single'", 1,"Single"]
data_dict.loc[52] = ["MaritalStatus","Marital status of the employee","2 'Married'", 2,"Married"]
data_dict.loc[53] = ["MaritalStatus","Marital status of the employee","3 'Divorced'", 3,"Divorced"]

##### MonthlyIncome & NumCompaniesWorked

In [145]:
gen_data['MonthlyIncome'].unique()

array([131160,  41890, 193280, ...,  37020,  23980,  54680])

In [146]:
gen_data['NumCompaniesWorked'].unique()

array([ 1.,  0.,  3.,  4.,  2.,  7.,  9.,  5.,  6.,  8., nan])

In [149]:
data_dict.loc[54,['Levels', 'level_grade','level_description']] = ["0 'Income'", 0,"Income"]
data_dict.loc[55,['Levels', 'level_grade','level_description']] = ["0 'Company_Count'", 0,"Company_Count"]


##### Over18

In [147]:
gen_data['Over18'].unique()

array(['Y'], dtype=object)

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=56, num_blank_rows=1)

In [154]:
data_dict.loc[56,['Levels', 'level_grade','level_description']] = ["1 'Y'", 1,"Y"]
data_dict.loc[57] = ["Over18","Whether the employee is above 18 years of age or not","2 'N'", 2,"N"]

##### PercentSalaryHike & StandardHours

In [148]:
gen_data['PercentSalaryHike'].unique()

array([11, 23, 15, 12, 13, 20, 22, 21, 17, 14, 16, 18, 19, 24, 25])

In [157]:
data_dict.loc[58,['Levels', 'level_grade','level_description']] = ["0 'Hike_percent'", 0,"Hike_percent"]

In [164]:
gen_data['StandardHours'].unique()

array([8])

In [171]:
data_dict.loc[67,['Levels', 'level_grade','level_description']] = ["0 'time_8hrs'", 0,"time_8hrs"]

##### StockOptionLevel

In [165]:
gen_data['StockOptionLevel'].unique()

array([0, 1, 3, 2])

In [ ]:
data_dict = insert_blank_rows_into_data_dict(data_dict, insert_index=68, num_blank_rows=3)

In [176]:
data_dict.loc[68,['Levels', 'level_grade','level_description']] = ["0 'Level_0'", 0,"Level_0"]
data_dict.loc[69] = ["StockOptionLevel","Stock option level of the employee","1 'Level_1'", 1,"Level_1"]
data_dict.loc[70] = ["StockOptionLevel","Stock option level of the employee","2 'Level_2'", 2,"Level_2"]
data_dict.loc[71] = ["StockOptionLevel","Stock option level of the employee","3 'Level_3'", 3,"Level_3"]

##### TotalWorkingYears & TrainingTimesLastYear & YearsAtCompany & YearsSinceLastPromotion & YearsWithCurrManager

In [166]:
gen_data['TotalWorkingYears'].unique()

array([ 1.,  6.,  5., 13.,  9., 28., 10., 21., 16., 37.,  7.,  3., 15.,
        8., nan, 12., 17., 19., 22.,  2.,  4., 23.,  0., 11., 24., 25.,
       20., 14., 26., 18., 30., 36., 31., 33., 32., 34., 40., 29., 35.,
       27., 38.])

In [167]:
gen_data['TrainingTimesLastYear'].unique()

array([6, 3, 2, 5, 4, 0, 1])

In [168]:
gen_data['YearsAtCompany'].unique()

array([ 1,  5,  8,  6,  7,  0,  9, 20, 15, 36, 10,  3, 17,  2,  4, 11, 22,
       18, 13, 24, 21, 16, 25, 29, 27, 14, 31, 32, 34, 26, 12, 19, 33, 30,
       23, 37, 40])

In [169]:
gen_data['YearsSinceLastPromotion'].unique()

array([ 0,  1,  7,  4, 10,  9,  6, 11,  3,  5,  2,  8, 13, 12, 15, 14])

In [170]:
gen_data['YearsWithCurrManager'].unique()

array([ 0,  4,  3,  5,  7,  8, 10, 11, 13,  9,  1,  2,  6, 12, 17, 16, 15,
       14])

In [178]:
data_dict.loc[72,['Levels', 'level_grade','level_description']] = ["0 'Experience'", 0,"Experience"]
data_dict.loc[73,['Levels', 'level_grade','level_description']] = ["0 'Training_time'", 0,"Training_time"]
data_dict.loc[78,['Levels', 'level_grade','level_description']] = ["0 'Tenure'", 0,"Tenure"]
data_dict.loc[79,['Levels', 'level_grade','level_description']] = ["0 'Years_since_promotion'", 0,"Years_since_promotion"]
data_dict.loc[80,['Levels', 'level_grade','level_description']] = ["0 'Years_with_manager'", 0,"Years_with_manager"]

In [182]:
# Convert to integers
data_dict['level_grade'] = data_dict['level_grade'].astype(int)

In [183]:
data_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Variable           81 non-null     object
 1   Meaning            81 non-null     object
 2   Levels             81 non-null     object
 3   level_grade        81 non-null     int64 
 4   level_description  81 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.3+ KB


#### General Data updates

In [181]:
gen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   object 
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   object 
 12  MaritalStatus            4410 non-null   object 
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [185]:
# Step 1: Create a dictionary mapping from 'level_description' to 'level_grade'
description_to_grade_mapping = data_dict.set_index('level_description')['level_grade'].to_dict()

# Step 2: Replace object column values in gen_data with the corresponding 'level_grade'
# Get all object columns in gen_data
object_columns = gen_data.select_dtypes(include='object').columns

# Step 3: For each object column, map its values to 'level_grade' based on 'level_description'
for col in object_columns:
    # Replace the values in the column using the description-to-grade mapping
    # Ensure original non-matching values remain intact (non-mapped values will stay as is)
    gen_data[col] = gen_data[col].map(description_to_grade_mapping).fillna(gen_data[col])


In [187]:
gen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   int64  
 2   BusinessTravel           4410 non-null   int64  
 3   Department               4410 non-null   int64  
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   int64  
 7   EmployeeCount            4410 non-null   int64  
 8   EmployeeID               4410 non-null   int64  
 9   Gender                   4410 non-null   int64  
 10  JobLevel                 4410 non-null   int64  
 11  JobRole                  4410 non-null   int64  
 12  MaritalStatus            4410 non-null   int64  
 13  MonthlyIncome            4410 non-null   int64  
 14  NumCompaniesWorked      

In [192]:
# Step 1: Fill NaN values in TotalWorkingYears with the corresponding values from YearsAtCompany
gen_data['TotalWorkingYears'] = gen_data['TotalWorkingYears'].fillna(gen_data['YearsAtCompany'])

# Step 2: Fill NaN values in NumCompaniesWorked based on the condition (TotalWorkingYears > YearsAtCompany)
condition = gen_data['TotalWorkingYears'] > gen_data['YearsAtCompany']
gen_data.loc[gen_data['NumCompaniesWorked'].isna() & condition, 'NumCompaniesWorked'] = 1

# Step 3: Fill remaining NaN values in NumCompaniesWorked with 0
gen_data['NumCompaniesWorked'] = gen_data['NumCompaniesWorked'].fillna(0)

In [197]:
# Step 4: Convert TotalWorkingYears and NumCompaniesWorked to integers
gen_data['TotalWorkingYears'] = gen_data['TotalWorkingYears'].astype(int)
gen_data['NumCompaniesWorked'] = gen_data['NumCompaniesWorked'].astype(int)

In [198]:
gen_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      4410 non-null   int64
 1   Attrition                4410 non-null   int64
 2   BusinessTravel           4410 non-null   int64
 3   Department               4410 non-null   int64
 4   DistanceFromHome         4410 non-null   int64
 5   Education                4410 non-null   int64
 6   EducationField           4410 non-null   int64
 7   EmployeeCount            4410 non-null   int64
 8   EmployeeID               4410 non-null   int64
 9   Gender                   4410 non-null   int64
 10  JobLevel                 4410 non-null   int64
 11  JobRole                  4410 non-null   int64
 12  MaritalStatus            4410 non-null   int64
 13  MonthlyIncome            4410 non-null   int64
 14  NumCompaniesWorked       4410 non-null   int64
 15  Over

### Manager Data

In [199]:
manager_data.head()


,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [200]:
manager_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   EmployeeID         4410 non-null   int64
 1   JobInvolvement     4410 non-null   int64
 2   PerformanceRating  4410 non-null   int64
dtypes: int64(3)
memory usage: 103.5 KB


### In Data

In [203]:
in_data


,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,...,NaN,2015-12-21 09:55:29,2015-12-22 10:04:06,2015-12-23 10:14:27,2015-12-24 10:11:35,NaN,2015-12-28 10:13:41,2015-12-29 10:03:36,2015-12-30 09:54:12,2015-12-31 10:12:44
1,2,NaN,2015-01-02 10:15:44,2015-01-05 10:21:05,NaN,2015-01-07 09:45:17,2015-01-08 10:09:04,2015-01-09 09:43:26,2015-01-12 10:00:07,2015-01-13 10:43:29,...,2015-12-18 10:37:17,2015-12-21 09:49:02,2015-12-22 10:33:51,2015-12-23 10:12:10,NaN,NaN,2015-12-28 09:31:45,2015-12-29 09:55:49,2015-12-30 10:32:25,2015-12-31 09:27:20
2,3,NaN,2015-01-02 10:17:41,2015-01-05 09:50:50,2015-01-06 10:14:13,2015-01-07 09:47:27,2015-01-08 10:03:40,2015-01-09 10:05:49,2015-01-12 10:03:47,2015-01-13 10:21:26,...,2015-12-18 10:15:14,2015-12-21 10:10:28,2015-12-22 09:44:44,2015-12-23 10:15:54,2015-12-24 10:07:26,NaN,2015-12-28 09:42:05,2015-12-29 09:43:36,2015-12-30 09:34:05,2015-12-31 10:28:39
3,4,NaN,2015-01-02 10:05:06,2015-01-05 09:56:32,2015-01-06 10:11:07,2015-01-07 09:37:30,2015-01-08 10:02:08,2015-01-09 10:08:12,2015-01-12 10:13:42,2015-01-13 09:53:22,...,2015-12-18 10:17:38,2015-12-21 09:58:21,2015-12-22 10:04:25,2015-12-23 10:11:46,2015-12-24 09:43:15,NaN,2015-12-28 09:52:44,2015-12-29 09:33:16,2015-12-30 10:18:12,2015-12-31 10:01:15
4,5,NaN,2015-01-02 10:28:17,2015-01-05 09:49:58,2015-01-06 09:45:28,2015-01-07 09:49:37,2015-01-08 10:19:44,2015-01-09 10:00:50,2015-01-12 10:29:27,2015-01-13 09:59:32,...,2015-12-18 09:58:35,2015-12-21 10:03:41,2015-12-22 10:10:30,2015-12-23 10:13:36,2015-12-24 09:44:24,NaN,2015-12-28 10:05:15,2015-12-29 10:30:53,2015-12-30 09:18:21,2015-12-31 09:41:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,NaN,2015-01-02 09:20:32,2015-01-05 10:17:53,2015-01-06 10:26:51,2015-01-07 10:06:58,2015-01-08 09:45:06,2015-01-09 09:49:24,2015-01-12 09:37:10,2015-01-13 09:25:02,...,2015-12-18 10:01:06,2015-12-21 10:25:25,2015-12-22 10:16:11,2015-12-23 10:04:40,2015-12-24 09:45:40,NaN,2015-12-28 10:15:39,2015-12-29 10:10:09,2015-12-30 09:28:19,2015-12-31 10:00:12
4406,4407,NaN,2015-01-02 10:03:41,NaN,2015-01-06 09:44:00,2015-01-07 09:42:10,2015-01-08 10:00:57,2015-01-09 09:44:04,2015-01-12 10:07:32,2015-01-13 10:05:11,...,2015-12-18 09:27:32,2015-12-21 09:41:24,2015-12-22 09:50:30,2015-12-23 10:32:21,2015-12-24 09:47:41,NaN,2015-12-28 09:54:23,2015-12-29 10:13:32,2015-12-30 10:21:09,2015-12-31 10:09:48
4407,4408,NaN,2015-01-02 10:01:01,2015-01-05 09:33:00,2015-01-06 09:49:17,2015-01-07 10:28:12,2015-01-08 09:47:38,2015-01-09 10:01:03,2015-01-12 09:49:12,2015-01-13 09:47:10,...,2015-12-18 10:00:57,2015-12-21 09:51:07,2015-12-22 10:02:10,2015-12-23 09:58:29,2015-12-24 09:56:05,NaN,2015-12-28 09:59:24,NaN,2015-12-30 10:02:36,2015-12-31 10:03:30
4408,4409,NaN,2015-01-02 10:17:05,2015-01-05 10:02:27,2015-01-06 10:12:50,2015-01-07 10:12:31,2015-01-08 09:42:57,NaN,2015-01-12 10:00:38,2015-01-13 09:48:03,...,2015-12-18 09:54:33,2015-12-21 10:01:08,2015-12-22 10:10:19,2015-12-23 09:42:30,2015-12-24 09:56:05,NaN,2015-12-28 09:55:25,2015-12-29 09:54:42,2015-12-30 10:15:44,2015-12-31 09:56:47


In [204]:
# Rename 'Unnamed: 0' to 'EmployeeID'
in_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

# Display the updated dataframe info to confirm the change
in_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 262 entries, EmployeeID to 2015-12-31
dtypes: float64(12), int64(1), object(249)
memory usage: 8.8+ MB


In [205]:
# Get all integer columns
int_columns = in_data.select_dtypes(include='int').columns
print("Integer Columns:", int_columns)

# Get all float columns
float_columns = in_data.select_dtypes(include='float').columns
print("Float Columns:", float_columns)


Integer Columns: Index(['EmployeeID'], dtype='object')
Float Columns: Index(['2015-01-01', '2015-01-14', '2015-01-26', '2015-03-05', '2015-05-01',
       '2015-07-17', '2015-09-17', '2015-10-02', '2015-11-09', '2015-11-10',
       '2015-11-11', '2015-12-25'],
      dtype='object')


In [206]:
# Convert EmployeeID column to integer type
in_data['EmployeeID'] = in_data['EmployeeID'].astype(int)

In [216]:
# Check for columns with NaN values
nan_columns = in_data.isna().sum()

# Filter columns that have NaN values
nan_columns = nan_columns[nan_columns > 0]
print(nan_columns)


Series([], dtype: int64)


In [211]:
# Convert all columns except 'EmployeeID' to datetime
for col in in_data.columns:
    if col != 'EmployeeID':
        in_data[col] = pd.to_datetime(in_data[col], errors='coerce')

# Verify the conversion
print(in_data.dtypes)


EmployeeID             int64
2015-01-01    datetime64[ns]
2015-01-02    datetime64[ns]
2015-01-05    datetime64[ns]
2015-01-06    datetime64[ns]
                   ...      
2015-12-25    datetime64[ns]
2015-12-28    datetime64[ns]
2015-12-29    datetime64[ns]
2015-12-30    datetime64[ns]
2015-12-31    datetime64[ns]
Length: 262, dtype: object


In [215]:
# Step 1: Replace NaT and other non-float-compatible values with 0.0
for col in in_data.columns:
    if col != 'EmployeeID':
        # Convert datetime columns to 0.0 if they contain NaT or non-float values
        in_data[col] = in_data[col].map(lambda x: 0.0 if pd.isna(x) or isinstance(x, pd.Timestamp) else x)

# Step 2: Convert all columns (except EmployeeID) to float
for col in in_data.columns:
    if col != 'EmployeeID':
        in_data[col] = in_data[col].astype(float)

# Verify the changes
print(in_data.dtypes)


EmployeeID      int64
2015-01-01    float64
2015-01-02    float64
2015-01-05    float64
2015-01-06    float64
               ...   
2015-12-25    float64
2015-12-28    float64
2015-12-29    float64
2015-12-30    float64
2015-12-31    float64
Length: 262, dtype: object


In [212]:
in_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 262 entries, EmployeeID to 2015-12-31
dtypes: datetime64[ns](261), int64(1)
memory usage: 8.8 MB


### Out Data

In [217]:
out_data.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 16:56:15,2015-01-05 17:20:11,2015-01-06 17:19:05,2015-01-07 16:34:55,2015-01-08 17:08:32,2015-01-09 17:38:29,2015-01-12 16:58:39,2015-01-13 18:02:58,...,NaN,2015-12-21 17:15:50,2015-12-22 17:27:51,2015-12-23 16:44:44,2015-12-24 17:47:22,NaN,2015-12-28 18:00:07,2015-12-29 17:22:30,2015-12-30 17:40:56,2015-12-31 17:17:33
1,2,NaN,2015-01-02 18:22:17,2015-01-05 17:48:22,NaN,2015-01-07 17:09:06,2015-01-08 17:34:04,2015-01-09 16:52:29,2015-01-12 17:36:48,2015-01-13 18:00:13,...,2015-12-18 18:31:28,2015-12-21 17:34:16,2015-12-22 18:16:35,2015-12-23 17:38:18,NaN,NaN,2015-12-28 17:08:38,2015-12-29 17:54:46,2015-12-30 18:31:35,2015-12-31 17:40:58
2,3,NaN,2015-01-02 16:59:14,2015-01-05 17:06:46,2015-01-06 16:38:32,2015-01-07 16:33:21,2015-01-08 17:24:22,2015-01-09 16:57:30,2015-01-12 17:28:54,2015-01-13 17:21:25,...,2015-12-18 17:02:23,2015-12-21 17:20:17,2015-12-22 16:32:50,2015-12-23 16:59:43,2015-12-24 16:58:25,NaN,2015-12-28 16:43:31,2015-12-29 17:09:56,2015-12-30 17:06:25,2015-12-31 17:15:50
3,4,NaN,2015-01-02 17:25:24,2015-01-05 17:14:03,2015-01-06 17:07:42,2015-01-07 16:32:40,2015-01-08 16:53:11,2015-01-09 17:19:47,2015-01-12 17:13:37,2015-01-13 17:11:45,...,2015-12-18 17:55:23,2015-12-21 16:49:09,2015-12-22 17:24:00,2015-12-23 17:36:35,2015-12-24 16:48:21,NaN,2015-12-28 17:19:34,2015-12-29 16:58:16,2015-12-30 17:40:11,2015-12-31 17:09:14
4,5,NaN,2015-01-02 18:31:37,2015-01-05 17:49:15,2015-01-06 17:26:25,2015-01-07 17:37:59,2015-01-08 17:59:28,2015-01-09 17:44:08,2015-01-12 18:51:21,2015-01-13 18:14:58,...,2015-12-18 17:52:48,2015-12-21 17:43:35,2015-12-22 18:07:57,2015-12-23 18:00:49,2015-12-24 17:59:22,NaN,2015-12-28 17:44:59,2015-12-29 18:47:00,2015-12-30 17:15:33,2015-12-31 17:42:14


In [218]:
out_data.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
out_data['EmployeeID'] = in_data['EmployeeID'].astype(int)

In [219]:
# Check for columns with NaN values
nan_columns = out_data.isna().sum()

# Filter columns that have NaN values
nan_columns = nan_columns[nan_columns > 0]
print(nan_columns)


2015-01-01    4410
2015-01-02     209
2015-01-05     206
2015-01-06     228
2015-01-07     209
              ... 
2015-12-25    4410
2015-12-28     234
2015-12-29     230
2015-12-30     265
2015-12-31     213
Length: 261, dtype: int64


In [220]:
# Convert all columns except 'EmployeeID' to datetime
for col in in_data.columns:
    if col != 'EmployeeID':
        in_data[col] = pd.to_datetime(in_data[col], errors='coerce')
        in_data[col] = in_data[col].map(lambda x: 0.0 if pd.isna(x) or isinstance(x, pd.Timestamp) else x)
        in_data[col] = in_data[col].astype(float)

# Verify the changes
print(in_data.dtypes)




EmployeeID      int64
2015-01-01    float64
2015-01-02    float64
2015-01-05    float64
2015-01-06    float64
               ...   
2015-12-25    float64
2015-12-28    float64
2015-12-29    float64
2015-12-30    float64
2015-12-31    float64
Length: 262, dtype: object


In [221]:
# Check for columns with NaN values
nan_columns = out_data.isna().sum()

# Filter columns that have NaN values
nan_columns = nan_columns[nan_columns > 0]
print(nan_columns)

2015-01-01    4410
2015-01-02     209
2015-01-05     206
2015-01-06     228
2015-01-07     209
              ... 
2015-12-25    4410
2015-12-28     234
2015-12-29     230
2015-12-30     265
2015-12-31     213
Length: 261, dtype: int64
